In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
import os
import pymatgen
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import trange, tqdm

In [3]:
structures = pd.read_csv("datasets/dichalcogenides_innopolis_202105/defects.csv", index_col=0)

In [4]:
defects_descriptors = pd.read_csv("datasets/dichalcogenides_innopolis_202105/descriptors.csv", index_col=0)

In [5]:
import pymatgen.io.cif
STRUCTURES_FOLDER = "datasets/dichalcogenides_innopolis_202105/initial/"
initial_structures = dict()
for structure_file in tqdm(os.listdir(STRUCTURES_FOLDER)):
  this_file = pymatgen.io.cif.CifParser(os.path.join(STRUCTURES_FOLDER, structure_file))
  initial_structures[os.path.splitext(structure_file)[0]] = this_file.get_structures()[0]

  0%|          | 0/3480 [00:00<?, ?it/s]

/temporary/pymatgen/pymatgen/io/cif.py:1123: UserWarning:

Issues encountered while parsing CIF: Some fractional co-ordinates rounded to ideal values to avoid issues with finite precision.



In [6]:
structures["initial_structure"] = structures.apply(lambda row: initial_structures[row._id], axis=1)

In [7]:
import matplotlib
import pylab

from math import floor, ceil
from random import shuffle, seed
from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from megnet.data.crystal import CrystalGraph
from megnet.data.molecule import MolecularGraph
from megnet.utils.preprocessing import StandardScaler
from megnet.callbacks import ModelCheckpointMAE
from pymatgen.core import Lattice, Structure, Molecule

cm = pylab.get_cmap('Set1')

import tensorflow as tf
import numpy as np

In [8]:
import wandb
from wandb.keras import WandbCallback

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train, test = train_test_split(structures, test_size=0.25, random_state=42)

In [11]:
wandb.init(project='ai4material_design', entity='kazeev')

wandb: Currently logged in as: kazeev (use `wandb login --relogin` to force relogin)


In [12]:
config = wandb.config
config.target = "homo"

In [13]:
gc = CrystalGraph(bond_converter=GaussianDistance(np.linspace(0, 5, 100), 0.5), cutoff=6)
model = MEGNetModel(nfeat_edge=100, nfeat_global=2, graph_converter=gc)#.from_file('8.9e-4')

In [14]:
scaler = StandardScaler.from_training_data(train["initial_structure"],
                                           train[config.target], is_intensive=True)
model.target_scaler = scaler

In [ ]:
model.train(train["initial_structure"], train[config.target],
            validation_structures=test["initial_structure"],
            validation_targets=test[config.target],
            callbacks=[WandbCallback()],
            epochs=100, verbose=1, patience=1000)

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_atom/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_atom/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_atom/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning:

Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/set2set_bond/Reshape_9:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/set2set_bond/Reshape_8:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/model/set2set_bond/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of m

21/21 [==============================] - 166s 7s/step - loss: 0.9786


INFO:megnet.callbacks:
Epoch 00001: val_mae improved from inf to 0.13076, saving model to callback/val_mae_00001_0.130756.hdf5


Epoch 2/100
21/21 [==============================] - 278s 13s/step - loss: 0.2467


...

Epoch 8/100
21/21 [==============================] - 248s 12s/step - loss: 0.0900
Epoch 9/100
21/21 [==============================] - 438s 21s/step - loss: 0.0899


INFO:megnet.callbacks:
Epoch 00009: val_mae improved from 0.07337 to 0.07324, saving model to callback/val_mae_00009_0.073241.hdf5


Epoch 10/100
21/21 [==============================] - 248s 12s/step - loss: 0.0910


INFO:megnet.callbacks:
Epoch 00010: val_mae improved from 0.07324 to 0.07231, saving model to callback/val_mae_00010_0.072305.hdf5


Epoch 11/100
21/21 [==============================] - 238s 12s/step - loss: 0.0925
Epoch 12/100
21/21 [==============================] - 279s 13s/step - loss: 0.0888
Epoch 13/100
21/21 [==============================] - 280s 13s/step - loss: 0.0846
Epoch 14/100
21/21 [==============================] - 315s 15s/step - loss: 0.0869
Epoch 15/100
21/21 [==============================] - 274s 13s/step - loss: 0.0851
Epoch 16/100
21/21 [==============================] - 406s 20s/step - loss: 0.0855
Epoch 17/100
21/21 [==============================] - 202s 10s/step - loss: 0.0845
Epoch 18/100
21/21 [==============================] - 99s 5s/step - loss: 0.0812
Epoch 19/100
21/21 [==============================] - 88s 4s/step - loss: 0.0862
Epoch 20/100
21/21 [==============================] - 71s 3s/step - loss: 0.0851
Epoch 21/100
21/21 [==============================] - 78s 4s/step - loss: 0.0839
Epoch 22/100
21/21 [==============================] - 77s 4s/step - loss: 0.0842
Epoch 23/100
2

In [ ]:
# TODO(kazeevn) speedup
test.loc[:, "predicted"] = model.predict_structures(test["initial_structure"].values)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(test[config.target], test["predicted"])
ax.set_xlabel("True energy per atom, eV")
ax.set_ylabel("Predicted energy per atom, eV");